<a href="https://colab.research.google.com/github/sheelapravalika/AI_CS_mokshasaireddy005/blob/main/SLM2_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sheelapravalika/AI_CS_mokshasaireddy005/blob/main/SLM2.ipynb)

In [ ]:
# ✅ Required Installs (if not done)
!pip install -U setfit datasets scikit-learn pandas matplotlib seaborn

In [ ]:
# ✅ Imports
from datasets import load_dataset, Dataset
from setfit import SetFitModel, SetFitTrainer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# ✅ Load CIC-IDS-2017 dataset
dataset = load_dataset("sonnh-tech1/cic-ids-2017", "binary")
df = dataset["train"].to_pandas()
print(df)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/92.8M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2522362 [00:00<?, ? examples/s]

         DestinationPort  FlowDuration  TotalFwdPackets  TotalBackwardPackets  \
0                  54865             3                2                     0   
1                  55054           109                1                     1   
2                  55055            52                1                     1   
3                  46236            34                1                     1   
4                  54863             3                2                     0   
...                  ...           ...              ...                   ...   
2522357               53         32215                4                     2   
2522358               53           324                2                     2   
2522359            58030            82                2                     1   
2522360               53       1048635                6                     2   
2522361               53         94939                4                     2   

         TotalLengthofFwdPa

In [ ]:
# ✅ Drop rows with NaNs
df = df.dropna()

In [ ]:
# ✅ Label encode
label_mapping = dict(enumerate(df["Label"].astype("category").cat.categories))
df["Label"] = df["Label"].astype("category").cat.codes

In [ ]:
# ✅ Select numeric features to embed into "sentence"
selected_features = [
    "Flow Duration", "Total Fwd Packets", "Total Backward Packets",
    "Total Length of Fwd Packets", "Total Length of Bwd Packets",
    "Fwd Packet Length Max", "Fwd Packet Length Min", "Bwd Packet Length Max", "Bwd Packet Length Min",
    "Flow Bytes/s", "Flow Packets/s", "Fwd IAT Total", "Bwd IAT Total"
]
selected_features = [col for col in selected_features if col in df.columns]

In [ ]:
# ✅ Convert features to sentence
df["sentence"] = df[selected_features].astype(str).agg(" ".join, axis=1)

In [ ]:
# ✅ Final DataFrame
final_df = df[["sentence", "Label"]].rename(columns={"Label": "label"})


In [ ]:
# ✅ Train/test split
train_df, test_df = train_test_split(final_df, test_size=0.2, stratify=final_df["label"], random_state=42)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

In [ ]:
# ✅ Load model
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L3-v2")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
# ✅ SetFit Trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class="softmax",
    batch_size=64,
    num_iterations=50,
    num_epochs=10,
    column_mapping={"sentence": "text", "label": "label"}
)

/tmp/ipython-input-801263738.py:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/2017889 [00:00<?, ? examples/s]

In [ ]:
# ✅ Train and measure time
import time
print("🚀 Starting SLM training...\n")
start_time = time.time()
trainer.train()
end_time = time.time()

🚀 Starting SLM training...



In [ ]:
# ✅ Display training duration
total_time = end_time - start_time
print(f"\n⏱️ Training completed in {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

# ✅ Evaluate
metrics = trainer.evaluate()
print(f"\n📊 Evaluation Accuracy: {metrics['accuracy']:.4f}")

# ✅ Predict and generate report
y_true = test_dataset["label"]
y_pred = trainer.model.predict(test_dataset["sentence"])

print("\n🧾 Classification Report:")
print(classification_report(y_true, y_pred, target_names=[label_mapping[i] for i in sorted(label_mapping.keys())]))

# ✅ Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
labels = [label_mapping[i] for i in sorted(label_mapping.keys())]

plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import nbformat

# Step 1: Upload your broken notebook again
from google.colab import files
uploaded = files.upload()

# Replace this with the actual uploaded filename
filename = list(uploaded.keys())[0]

# Step 2: Load and clean
with open(filename, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# ✅ Step 3: Clean ONLY metadata.widgets (KEEP outputs and execution_count)
for cell in nb.cells:
    if 'metadata' in cell:
        cell['metadata'] = {k: v for k, v in cell['metadata'].items() if k != 'widgets'}

# ✅ Step 4: Clear notebook-level metadata.widgets (if present)
if 'widgets' in nb.get('metadata', {}):
    del nb['metadata']['widgets']

# Step 5: Save cleaned notebook
cleaned_filename = filename.replace('.ipynb', '_fixed.ipynb')
with open(cleaned_filename, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print(f"✅ Cleaned notebook saved (outputs preserved) as {cleaned_filename}")
files.download(cleaned_filename)